# These sets of models all calcuate the compliacne of a part.
Uses:
    - The part data, a 2D binary array representing where mass is present.
    - The Force vectors, 2 2D arrays representing the forces baing applied to the part 
        - One array holds the forces in the x direction
        - The other array holds the forces in the y direction
    - The supports, The circle where the part is anchored and doesn't move
    - The maxCompliance. 
        - Used only by the classifier model(discontinued)
Returns:
    - The classifier would return a boolean value if the compliance of the element was witin the max compliance constraint
    - The reggressor model would retunr the log(natural) of the compliance
    

In [ ]:
#data management libraries
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# model building libraries
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# library for gathering data
import os

The cell below gets the data path loaded into memmory.
It then counts the number of avalible data points

In [ ]:
DATA_FILE_PATH= os.path.join(os.path.dirname(os.getcwd()),r"DataGathering\Data\30_30")
print(DATA_FILE_PATH)

dir_list = os.listdir(DATA_FILE_PATH)
max_data_points = len(dir_list)
print("Number of data points: {}".format(len(dir_list)))
print(dir_list[0])

In [ ]:
def getAgent(path):
    """
    Returns the uncompressed numpy arrays inside a file pointed to by path
    The path must point to a file FOLDER the has one or more npz files in it.

    This will only properly return values for npz files saved with the following data(see the data creation algorithms).
        - xPhys
        - forces
        - degreesOfFreedom
        - formating_array
    """
    filesToPull = os.listdir(path)

    dataPoints = []

    for fileName in filesToPull:
        try:

            data = np.load(os.path.join(path,fileName),allow_pickle=True)

            xPhys = data['a']
            forces = data['b']
            degreesOfFreedom = data['c']
            formating_array = data['d']

            maxDof = forces.shape[0]
            toPad = maxDof - len(degreesOfFreedom)
            degreesOfFreedom = np.pad(degreesOfFreedom,[0,toPad])

            compliance = formating_array[0]
            compliance_max = formating_array[1]
            shape = formating_array[2:]

            #forces = np.reshape(forces,(shape[0],shape[1],4))

            dataPoints.append([xPhys,forces,degreesOfFreedom,compliance_max,shape,compliance])

        except:
            print("Error in reading file couldn't get file: {}".format(fileName))
    
    return dataPoints




In [ ]:
#get some basic information about the shapes of the data
d1 = getAgent(os.path.join(DATA_FILE_PATH,dir_list[0]))

print("xPhys.shape:",d1[0][0].shape)
print("forces.shape:",d1[0][1].shape)
print("dofs.shape:",d1[0][2].shape)
print("c-Max:",d1[0][3])
print("compliance:",d1[0][5])
print("shape:",d1[0][4])

shape_of_Inputs = d1[0][4]

In [ ]:
def formatData(xPhys,forces,degreesOfFreedom,compliance_max,shape):
    """
    Given the data from the compressed NPZ file return an array of the data formatted as a series of 2D arrays(images) all of the same size
    """
    #since xphys is only nelx by nely get the size difference between it and thepropper shape (should be 1)
    finalShape = (int(shape[0]+1),int(shape[1]+1))
    xPhysShape = xPhys.shape
    difference = (int(finalShape[0]-xPhysShape[0]),int(finalShape[1]-xPhysShape[1]))
    # pad xPhys to be the porpper size of the rest of the images
    xPhys2 = np.pad(xPhys,[[0,difference[0]],[0,difference[1]]])

    # the 2D forces are 2 arrays of size 2*(nelx+1)*(nely+1)
    # The two arrays represent the two individual forces being applied to the part
    # sumt the forces along the second axis (not 0) in order to get the two sets of forces on a single array.
    forces2 = forces.sum(1)

    #reshape the forces array into 2 arrays of propper size (one for x, one for y)
    forces2 = np.reshape(forces2,(finalShape[0],finalShape[1],2))

    # degrees of freedom is a list of indexes represening all the elements of the part that can move(have displacement thus be compliant)
    # the following code creates and array where all indexes where the part can move are 0 and all anchored elements(supports) are 1
    d2 = np.ones(2*finalShape[0]*finalShape[1])
    for index in degreesOfFreedom:
        d2[index] = 0

    # since this follows the same rule as the forces with respect to the x and y direction it must be two arrays
    d3 = np.reshape(d2,(finalShape[0],finalShape[1],2))
    # since it will be anchored fully(in both x and y directions) we will sum it to compress the two arrays into one
    degreesOfFreedom2 = d3.sum(2)

    #a 2D array of the compliance as an image
    c_max = compliance_max * np.ones((finalShape[0],finalShape[1]))

    #previously, each of the inputs were stacked together into a (nelx+1) by (nely+1) by 5 matrix that would be passed to a convolution
    # now each entry is returned in a list to allow easy seperation
    arraysToStack = [xPhys2,forces2[:,:,0],forces2[:,:,1],degreesOfFreedom2]
    fullPart = np.stack(arraysToStack,axis=-1).astype('float32')

    return [xPhys2,forces2,degreesOfFreedom2,compliance_max]

    

In [ ]:
#get the shape of the formatted data
for xPhys,forces,degreesOfFreedom,compliance_max,shape,compliance in getAgent(os.path.join(DATA_FILE_PATH,dir_list[0])):
    data = formatData(xPhys,forces,degreesOfFreedom,compliance_max,shape)
    print(data[0].shape)
    print(data[1].shape)

Here, the data is brought from each of the files and loaded into memory.

You can define you own number of points to use as an integer > 0. This will then be limited to the max allowable number of points defeined above.

Data is formated as follows
- Data_x is a list holding the formated data(see above)
- Data_y is a numpy array holding the coresponding compliance values for each part
    - Since compliance can range from +0 to +inf we take the natural log of it to allow the model to reuturn a smaller number from -inf to +inf.


More data is needed to train more accurate models.
This data could be gathered by rotating/fliping the formated data. Care must be taken to ensure all points still line up correctly

In [ ]:
#Get the data
points_to_Use = 100000
"""
1000    = 11 seconds
10000   = 4 minutes
"""
points_to_Use = min(points_to_Use,max_data_points)

# can use an np.arrange and np.random.shuffle to get random data points every time
pointsShuffeled = np.arange(max_data_points).astype("int32")
np.random.shuffle(pointsShuffeled)

badData = 0


data_x = []
data_y = []
for i in range(points_to_Use):
    data1 = getAgent(os.path.join(DATA_FILE_PATH,dir_list[pointsShuffeled[i]]))
    for xPhys,forces,degreesOfFreedom,compliance_max,shape,compliance in data1: 
        x = formatData(xPhys,forces,degreesOfFreedom,compliance_max,shape)
        data_x.append(x)
        y_val = np.log(compliance).astype('float32')
        data_y.append(y_val)
        if(y_val <= 0):
            #print(i,compliance)
            badData += 1
            data_x.pop()
            data_y.pop()

#data_x = np.array(data_x)
data_y = np.array(data_y)

print(len(data_x))
print(len(data_y))
print("Bad data:",badData)

In [ ]:
#get some statistics on the classification data(deprecieated)
numTrue = 0
numFalse = 0

numTrue = int(data_y.sum())
numFalse = len(data_y) - numTrue

print(numTrue)
print(numFalse)

In [ ]:
#get some statistics on the regression data

yMax = np.max(data_y)
yMin = np.min(data_y)

print(yMax)
print(yMin)

In [ ]:
#perform a test train split
X_train, X_test, Y_train, Y_test  = train_test_split(data_x,data_y, test_size=0.25, random_state=42)
print("X_train: {}\nY_train: {}".format(len(X_train), len(Y_train)))
print("X_test: {}\nY_test: {}".format(len(X_test), len(Y_test)))

The following code seperates the test train split data into groups of the seperate part images

A seperate array is formed for each of the images
A for loop extracts the images and puts them into seperate arrays
The arrays are kept in the order that is defined by the test train split

In [ ]:
X_train_part = []
X_train_compliance = []
X_train_forces=[]
X_train_supports = []

X_test_part = []
X_test_compliance = []
X_test_forces=[]
X_test_supports = []

for x,f,dof,compliance_max in X_train:
    X_train_part.append(x)
    X_train_forces.append(f)
    X_train_supports.append(dof)
    X_train_compliance.append(compliance_max)

for x,f,dof,compliance_max in X_test:
    X_test_part.append(x)
    X_test_forces.append(f)
    X_test_supports.append(dof)
    X_test_compliance.append(compliance_max)

X_train_part = np.array(X_train_part)
X_train_forces = np.array(X_train_forces)
X_train_supports = np.array(X_train_supports)
X_train_compliance = np.array(X_train_compliance)

X_test_part = np.array(X_test_part)
X_test_forces = np.array(X_test_forces)
X_test_supports = np.array(X_test_supports)
X_test_compliance = np.array(X_test_compliance)


Here, we define the model constants as well as the path where the model will be saved.
Once the model completes an epoch it will be saved to the defined path

The input shape will be the shape of the stacked part images(unused in later iteration of the model)
activation is defined as 'relu' since it is easy and fast
the uniform initalizer is unused after little progress was show when using it

In [ ]:
ToSavePath = os.path.join(os.path.dirname(os.getcwd()),r"Model Creation\models\complianceRegressor4")

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=str(ToSavePath),
                                                     save_weights_only=True,
                                                     verbose=1)

InputShape = (31,31,4)
activation = 'relu'
uniformRandomInitalizer = tf.random_uniform_initializer(minval=-0.1, maxval=0.5)

the model will have a generic input of a 31 by 31 image with 4 channels representing the density grid, the forces in both the x and y direction, and the supports. A 5th channel, holding a matrix of compliance max values, will be stacked on the layers as they pass through each convolution. The same value will be concatenated in each dense layer once we flatten it.

The reutnr of the models is a boolean value retpresenting if the given part is within compliance.

The actual inputs differ by model some take the stacked part images, some take the part images seperately

In [ ]:
def m1_classification():
    partInput = keras.Input(shape=InputShape,name="partImage")
    complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    x1 = layers.Conv2D(filters= 16,kernel_size=(3,3))(partInput)
    cScaling = layers.Reshape((31,31,1))(layers.RepeatVector(31*31)(complianceMaxInput))

    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x2 = layers.MaxPooling2D(pool_size=(2,2))(x1)

    x2 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x3 = layers.MaxPooling2D(pool_size=(2,2))(x2)

    x_flat = layers.Flatten()(x3)
    x4 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(31,activation=activation)(x4)

    x4 = layers.concatenate([x4,complianceMaxInput])
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,complianceMaxInput],outputs=outputs)

In [ ]:
def m2_classification():
    partInput = keras.Input(shape=InputShape,name="partImage")
    complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    x1 = layers.Conv2D(filters= 16,kernel_size=(3,3))(partInput)
    cScaling = layers.Reshape((31,31,1))(layers.RepeatVector(31*31)(complianceMaxInput))

    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    x2 = layers.Concatenate()([x1,cScaling])
    x2 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2))(x2)

    x_flat = layers.Flatten()(x2)
    x3 = layers.concatenate([x_flat,complianceMaxInput])
    x3 = layers.Dense(31,activation=activation)(x3)

    x4 = layers.concatenate([x3,complianceMaxInput])
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,complianceMaxInput],outputs=outputs)

In [ ]:
def m3_classification():
    partInput = keras.Input(shape=(31,31,1),name="partImage")
    forceInput = keras.Input(shape=(31,31,2),name="Forces")
    supportInput = keras.Input(shape=(31,31,1),name="Supports")
    complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    partConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(partInput)
    forceConv = layers.Conv2D(filters= 10,kernel_size=(3,3),padding='same',activation='tanh')(forceInput)
    supportConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(supportInput)
    cScaling = layers.Reshape((supportConv.shape[1],supportConv.shape[2],1))(layers.RepeatVector(supportConv.shape[1]*supportConv.shape[2])(complianceMaxInput))


    concatenatedConvolution = layers.Concatenate()([partConv,forceConv,supportConv,cScaling])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(concatenatedConvolution)
    x1 = layers.Dropout(0.1)(x1)
    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    #partScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(concatenatedConvolution)
    x2 = layers.Concatenate()([x1,cScaling])
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Dropout(0.3)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x2)

    newShape = (x2.shape[1],x2.shape[2])
    #print(x1.shape)
    cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    x3 = layers.Concatenate()([x2,cScaling])
    x3 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x3)

    x_flat = layers.Flatten()(x3)
    #x3 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(32,activation=activation)(x_flat)
    #x3 = layers.Dropout(0.2)(x3)

    #x4 = layers.concatenate([x3,complianceMaxInput])
    #outputs = layers.GlobalAveragePooling1D()(x4)
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,forceInput,supportInput,complianceMaxInput],outputs=outputs)


When little progress was made with classification model I moved to regression.

The regession models take the sperate part images as seperate inputs and return the natural log of the compliance

In [ ]:
def m4_regression():
    partInput = keras.Input(shape=(31,31,1),name="partImage")
    forceInput = keras.Input(shape=(31,31,2),name="Forces")
    supportInput = keras.Input(shape=(31,31,1),name="Supports")
    #complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    partConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(partInput)
    forceConv = layers.Conv2D(filters= 10,kernel_size=(3,3),padding='same',activation='tanh')(forceInput)
    supportConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(supportInput)
    #cScaling = layers.Reshape((supportConv.shape[1],supportConv.shape[2],1))(layers.RepeatVector(supportConv.shape[1]*supportConv.shape[2])(complianceMaxInput))


    concatenatedConvolution = layers.Concatenate()([partConv,forceConv,supportConv])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(concatenatedConvolution)
    x1 = layers.Dropout(0.1)(x1)
    #x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    partScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(concatenatedConvolution)
    x2 = layers.Concatenate()([x1,partScaling])
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Dropout(0.3)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x2)

    newShape = (x2.shape[1],x2.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    #x3 = layers.Concatenate()([x2,cScaling])
    x3 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x3 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.Dropout(0.3)(x3)
    x3 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x3)

    x_flat = layers.Flatten()(x3)
    #x3 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(32,activation=activation)(x_flat)
    #x3 = layers.Dropout(0.2)(x3)

    #x4 = layers.concatenate([x3,complianceMaxInput])
    #outputs = layers.GlobalAveragePooling1D()(x4)
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,forceInput,supportInput],outputs=outputs)


In [ ]:
def m5_regression():
    partInput = keras.Input(shape=(31,31,1),name="partImage")
    forceInput = keras.Input(shape=(31,31,2),name="Forces")
    supportInput = keras.Input(shape=(31,31,1),name="Supports")
    #complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    partConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(partInput)
    forceConv = layers.Conv2D(filters= 10,kernel_size=(3,3),padding='same',activation='tanh')(forceInput)
    supportConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(supportInput)
    #cScaling = layers.Reshape((supportConv.shape[1],supportConv.shape[2],1))(layers.RepeatVector(supportConv.shape[1]*supportConv.shape[2])(complianceMaxInput))


    concatenatedConvolution = layers.Concatenate()([partConv,forceConv,supportConv])
    x1 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(concatenatedConvolution)
    x1 = layers.GaussianNoise(stddev=1)(x1)
    x1 = layers.ReLU()(x1)
    x1 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Dropout(0.3)(x1)
    x1 = layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    partScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(concatenatedConvolution)
    x2 = layers.Concatenate()([x1,partScaling])
    x2 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.GaussianNoise(stddev=1)(x2)
    x2 = layers.ReLU()(x2)
    x2 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Dropout(0.3)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x2)

    newShape = (x2.shape[1],x2.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    #x3 = layers.Concatenate()([x2,cScaling])
    x3 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x3 = layers.GaussianNoise(stddev=1)(x3)
    x3 = layers.ReLU()(x3)
    x3 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.Dropout(0.3)(x3)
    x3 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x3)

    x_flat = layers.Flatten()(x3)
    #x3 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(32,activation=activation)(x_flat)
    #x3 = layers.Dropout(0.2)(x3)

    #x4 = layers.concatenate([x3,complianceMaxInput])
    #outputs = layers.GlobalAveragePooling1D()(x4)
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,forceInput,supportInput],outputs=outputs)


In [ ]:
def m6_regression():
    partInput = keras.Input(shape=(31,31,1),name="partImage")
    forceInput = keras.Input(shape=(31,31,2),name="Forces")
    supportInput = keras.Input(shape=(31,31,1),name="Supports")
    #complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    partConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(partInput)
    forceConv = layers.Conv2D(filters= 10,kernel_size=(3,3),padding='same',activation='tanh')(forceInput)
    supportConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(supportInput)
    #cScaling = layers.Reshape((supportConv.shape[1],supportConv.shape[2],1))(layers.RepeatVector(supportConv.shape[1]*supportConv.shape[2])(complianceMaxInput))


    concatenatedConvolution = layers.Concatenate()([partConv,forceConv,supportConv])
    x1 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(concatenatedConvolution)
    x1 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.MaxPool2D(pool_size=(3,3),strides=(3,3),padding='same')(x1)

    newShape = (x1.shape[1],x1.shape[2])
    partScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(concatenatedConvolution)
    x2 = layers.Concatenate()([x1,partScaling])
    x2 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.MaxPooling2D(pool_size=(3,3),strides=(3,3),padding='same')(x2)

    x_flat = layers.Flatten()(x2)
    x4 = layers.Dense(x_flat.shape[1],activation=activation)(x_flat)
    x4 = layers.Dense(31,activation=activation)(x4)

    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,forceInput,supportInput],outputs=outputs)


In [ ]:
#here we build and create a summary of the model we wish to train.

comp_tester = m6_regression()
comp_tester.summary()
#keras.utils.plot_model(comp_tester, os.path.join(os.getcwd(),"model1.png"), show_shapes=True)

In [ ]:
"""
Little testing has gon into the difference between optimizers.

The loss is the standard loss function for regression

If you wish to load a model then use the load weights function otherwise comment it out
Note that models are not named so inf you save a model you may want to actually name it (see above with the mode save path).
"""

selfMadOptomizer = tf.keras.optimizers.Adam(learning_rate=0.001)
comp_tester.compile(optimizer=selfMadOptomizer, loss=tf.keras.losses.MeanSquaredError())
#comp_tester.load_weights(str(ToSavePath))
# comp_tester.compile(optimizer=selfMadOptomizer,
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [ ]:
# fit the model to the training data.
comp_tester.fit(
                {"partImage":X_train_part, "Forces":X_train_forces, "Supports":X_train_supports},#, "complianceMax":X_train_compliance},
                Y_train,
                epochs=5,
                shuffle=True,
                validation_data=(
                    {"partImage":X_test_part, "Forces":X_test_forces, "Supports":X_test_supports},#,"complianceMax":X_test_compliance}, 
                        Y_test),
                callbacks=[cp_callback])

Once the model has been trained we can run some stats on it.
The following cells do just that however they use testing data, which while the model theoretically has "never seen" the data it is still the data it was tested on

More data is needed to create a set of random data that can be used solely for this stats run.

In [ ]:
pointsToTest = 100
dataSample_part = X_test_part[pointsToTest:]
dataSample_forces = X_test_forces[pointsToTest:]
dataSample_supports = X_test_supports[pointsToTest:]

dataSample_Y_True = Y_test[pointsToTest:]

dataSample_Y_pred = comp_tester.predict({"partImage":dataSample_part, "Forces":dataSample_forces, "Supports":dataSample_supports})


In [ ]:
errorList = []

for i in range(pointsToTest):
    #print(dataSample_Y_True[i],dataSample_Y_pred[i][0])
    a = np.exp(dataSample_Y_True[i])
    b = np.exp(dataSample_Y_pred[i][0])
    #standard relative error
    if(a != 0):
        errorList.append((abs(a-b))/a)

error = np.array(errorList)

print("mean:", np.mean(error))
print("standard deviation:", np.std(error))
print("min: {}\tmax: {}".format(np.min(error),np.max(error)))
    

In [ ]:
for i in range(10):
    #print(dataSample_Y_True[i],dataSample_Y_pred[i][0])
    a = np.exp(dataSample_Y_True[i])
    b = np.exp(dataSample_Y_pred[i][0])
    print("True: {:.3f}\t Predicted: {:.3f}\t error: {:.3f}".format(a,b,((a-b))/a))

# Thoughts

The regression model works well however it is prone to random cases where it is very incorrect. If adding more data to the mix fails to reduce this we can boost the model. Ideally we could use multiple smaller moddels to perform the same calculations and the we can average their return values to hopefully get accurate data without the outlining random error spikes.